---
# --- Day 19: Aplenty ---
---

In [134]:
from typing_extensions import TypedDict
from typing import Optional, List
import re
import numpy as np

## Load data

In [54]:
full_puzzle_data = False

In [55]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day19_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [56]:
class MachinePart(TypedDict):
    x: int
    m: int
    a: int
    s: int

In [57]:
class Rule:
    def __init__(self, rule_def: str):
        condition, self.outcome = rule_def.split(":")
        self.part_id = condition[0]
        self.multiplier = 1 if condition[1] == ">" else -1
        self.quantity = int(condition[2:])
        
    def execute_rule(self, part: MachinePart) -> Optional[str]:
        if (part[self.part_id] - self.quantity) * self.multiplier > 0:
            return self.outcome
        else:
            return None        

In [58]:
class Workflow:
    def __init__(self, rules_string: str):
        rules_data = rules_string.split(",")
        self.rules = [Rule(r) for r in rules_data[:-1]]
        self.final_outcome = rules_data[-1]        
    
    def execute_workflow(self, part: MachinePart) -> str:
        for r in self.rules:
            rule_outcome = r.execute_rule(part)
            if rule_outcome:
                return rule_outcome
        return self.final_outcome

In [59]:
workflows = dict()
for i, row in enumerate(data):
    if not row:
        i += 1
        break
    wid, rules_string = row.split("{")
    workflows[wid] = Workflow(rules_string[:-1])

In [60]:
parts = []
for row in data[i:]:
    v = list(map(int, re.findall(r"\d+", row)))
    parts.append(MachinePart(x=v[0],m=v[1],a=v[2],s=v[3]))

## --- Part One ---

In [61]:
sum_accepted = 0
for p in parts:
    out = workflows["in"].execute_workflow(p)
    while not out in ["A", "R"]:
        out = workflows[out].execute_workflow(p)
    if out == "A":
        sum_accepted += sum(p.values())

In [62]:
print(sum_accepted)

19114


## --- Part Two ---

#### create Workflow dictionary differently

In [128]:
def reverse_inequality(s: str):
    signs = [">", "<"]
    i  = int("<" in s)
    part, qnt = s.split(signs[i])
    qnt = int(qnt) + 1 if i == 0 else int(qnt) - 1
    return part + signs[1-i] + str(qnt)
    
WF = dict()
for i, row in enumerate(data):
    if not row:
        i += 1
        break
    wid, rules_string = row.split("{")
    conditions = rules_string[:-1].split(",")
    wf_map = {c.split(":")[0]: c.split(":")[1] for c in conditions[:-1]}
    complementary = " and ".join([reverse_inequality(k) for k in wf_map.keys()])
    wf_map[complementary] = conditions[-1]
    WF[wid] = wf_map

In [129]:
def gather_conditions(workflow_id: str):
    if workflow_id in ["A", "R"]:
        return [workflow_id]
    else:
        return [[k] + gather_conditions(v) for k, v in WF[workflow_id].items()]    

In [130]:
def get_paths(d, c = []):
    for a, *b in d:
        if len(b) == 1 and not isinstance(b[0], list):
            yield (b[0], " and ".join(c+[a]))
        else:
            yield from get_paths(b, c+[a])

In [131]:
conditions = gather_conditions("in")
valid_paths = [p[1] for p in get_paths(conditions) if p[0] == "A"]

In [229]:
def find_valid_support(cond: str) -> np.ndarray:
    n = 4000
    parts = ["x", "m", "a", "s"]
    signs = [">", "<"]
    support = np.ones((len(parts), n), dtype=int)
    ineqs = cond.split(" and ")
    for c in ineqs:
        isn  = int("<" in c)
        part, qnt = c.split(signs[isn])
        qnt = int(qnt) - 1
        pid = np.where(np.array(parts)==part)[0][0]
        if isn == 0:
            support[pid, :qnt+1] = 0
        else:
            support[pid, qnt:] = 0
    return support    

In [244]:
support = np.zeros((4000, 4000, 4000, 4000), dtype=bool)
n = 0
for p in valid_paths:
    ps = find_valid_support(p)
    ps = np.clip(ps - support, 0, 1)
    n += np.prod(np.sum(ps, axis=1))
    support |= ps

MemoryError: Unable to allocate 233. TiB for an array with shape (4000, 4000, 4000, 4000) and data type bool

In [243]:
valid_paths

['s<1351 and a<2006 and x<1416',
 's<1351 and a<2006 and x>1415 and x>2662',
 's<1351 and m>2090',
 's<1351 and a>2005 and m<2091 and s>536 and x<2441',
 's>1350 and s>2770 and s>3448',
 's>1350 and s>2770 and s<3449 and m>1548',
 's>1350 and s>2770 and s<3449 and m<1549',
 's>1350 and m<1801 and m>838',
 's>1350 and m<1801 and m<839 and a<1717']